<a href="https://colab.research.google.com/github/lucascarter0/DeepLearning.AI-TensorFlow-Certification/blob/main/CNN_Multiclass_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Notebook for Week 4 of the Convolutional Neural Network course - notebook trains a multi-class classifier on the MNIST digit database.

In [20]:
import csv
from os import getcwd
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from keras.datasets import mnist

In [21]:
# Load MNIST data and format it for training in the model
(training_images, training_labels), (testing_images, testing_labels) = mnist.load_data()
training_images = np.expand_dims(training_images, axis=3)
testing_images = np.expand_dims(testing_images, axis=3)

In [22]:
# Create an ImageDataGenerator to do image augmentation on training set
train_datagen = ImageDataGenerator(
      rescale = 1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

# Validation data also using generator, but only to scale image color
validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow(training_images, training_labels, batch_size=20)

validation_generator = validation_datagen.flow(testing_images, testing_labels, batch_size=20)


In [27]:
# Create a model with two convolution layers and two pooling layers
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(26, activation='softmax')
    ])

# Compile Model. Using the sparse crossentropy loss for the multiclass data
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=RMSprop(learning_rate=0.001),
              metrics=['accuracy'])

In [25]:
# Define a Callback class that stops training once accuracy reaches 98.0%
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy') > 0.97):
      print("\nReached 98.0% accuracy, cancelling training")
      self.model.stop_training = True

In [ ]:
# Train and evaluate model - train on 10 epochs, but model will exit at 98% accuracy
history = model.fit(train_generator,
                    validation_data=validation_generator,
                    epochs=10,
                    callbacks=[myCallback()])

model.evaluate(testing_images, testing_labels, verbose=0)

Epoch 1/10
3000/3000 [==============================] - 89s 29ms/step - loss: 0.9122 - accuracy: 0.6919 - val_loss: 0.2484 - val_accuracy: 0.9237
Epoch 2/10
3000/3000 [==============================] - 88s 29ms/step - loss: 0.4418 - accuracy: 0.8598 - val_loss: 0.1429 - val_accuracy: 0.9536
Epoch 3/10
3000/3000 [==============================] - 87s 29ms/step - loss: 0.3511 - accuracy: 0.8905 - val_loss: 0.1431 - val_accuracy: 0.9546
Epoch 4/10
3000/3000 [==============================] - 87s 29ms/step - loss: 0.3182 - accuracy: 0.9025 - val_loss: 0.1665 - val_accuracy: 0.9520
Epoch 5/10
3000/3000 [==============================] - 87s 29ms/step - loss: 0.3174 - accuracy: 0.9039 - val_loss: 0.1343 - val_accuracy: 0.9627
Epoch 6/10
3000/3000 [==============================] - 86s 29ms/step - loss: 0.3149 - accuracy: 0.9047 - val_loss: 0.1874 - val_accuracy: 0.9554
Epoch 7/10
2586/3000 [========================>.....] - ETA: 11s - loss: 0.3184 - accuracy: 0.9043

In [ ]:
%matplotlib inline
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

fig, (ax0, ax1) = plt.subplots(nrows=2, figsize=(10, 10))
ax0.plot(epochs, acc, 'r', label='Training accuracy')
ax0.plot(epochs, val_acc, 'b', label='Validation accuracy')
ax0.title('Training and validation accuracy')
ax0.legend()

ax1.plot(epochs, loss, 'r', label='Training Loss')
ax1.plot(epochs, val_loss, 'b', label='Validation Loss')
ax1.title('Training and validation loss')
ax1.legend()

#plt.show()